In [29]:
def transformar_labels_e_exit_em_linhas(asm_codigo: str) -> str:
    linhas = asm_codigo.splitlines()
    mapeamento_labels = {}
    resultado_asm = []

    # Determinar a localização dos rótulos e do "exit"
    for indice, linha in enumerate(linhas, start=1):
        linha = linha.strip()
        if linha.endswith(":"):
            label = linha[:-1]  # Captura o nome da label sem os dois pontos
            mapeamento_labels[label] = indice  # Mapear o rótulo para o número da linha
        elif "exit" in linha:
            mapeamento_labels["exit"] = indice  # Adiciona o mapeamento para exit

    cont=0
    # Substituir os rótulos e o "exit" por números de linha
    for linha in linhas:
        cont+=1
        linha_formatada = linha
        for label, numero_linha in mapeamento_labels.items():
            if f", {label}" in linha:
                linha_formatada = linha_formatada.replace(f", {label}", f", {numero_linha-cont-1}")
            #if f"{label}" in linha and linha.strip() == "exit":  # Substitui o "exit" pelo número correspondente
            #    linha_formatada = linha_formatada.replace("exit", str(numero_linha))
        if not linha_formatada.endswith(":"):  # Ignorar linhas que são apenas rótulos
            resultado_asm.append(linha_formatada)

    return "\n".join(resultado_asm)

# Exemplo de uso
entrada_asm = """mov %r2, 0xe
ldxh %r3, [%r1+12]
jne %r3, 0x81, L1
mov %r2, 0x12
ldxh %r3, [%r1+16]
and %r3, 0xffff
L1:
jne %r3, 0x8, L2
add %r1, %r2
mov %r0, 0x1
ldxw %r1, [%r1+16]
and %r1, 0xffffff
jeq %r1, 0x1a8c0, exit
L2:
mov %r0, 0x0
exit"""

saida_asm = transformar_labels_e_exit_em_linhas(entrada_asm)
#print(saida_asm)

In [30]:
import os

def parseTests(conteudo_data: str) -> str:
    # Separar as seções da string de entrada
    cabecalho = """import «ProjetoConformance».ConformanceTests
"""
    cabecalhoOld = """import «ProjetoConformance».ConformanceTests
    structure MemorySpace where
    data : Fin 512 → Nat \n
def emptyMemory : MemorySpace := {data := fun _ => 0}
"""
    
    partes = conteudo_data.split("--")
    mem_secao = None
    asm_secao = None
    result_secao = None

    for parte in partes:
        if parte.strip().startswith("mem"):
            mem_secao = parte.split("\n", 1)[1].strip()  # Obter apenas o conteúdo após "mem"
        elif parte.strip().startswith("asm"):
            asm_secao = parte.split("\n", 1)[1].strip()  # Obter a parte ASM
        elif parte.strip().startswith("result"):
            result_secao = parte.split("\n", 1)[1].strip()  # Obter o conteúdo do resultado
    
    if not mem_secao :
        mem_secao = "00 00 00 00 00 00 00 00"

    # Criar o formato .lean
    #if not mem_secao or not asm_secao or not result_secao:
        #return "Erro: Seções necessárias não foram encontradas no arquivo de entrada."
    

    # Formatar a memória para o formato lean
    mem_formatada = "{mem|\n" + "\n".join(
        [" ".join(f"'{char}'" for char in linha.strip() if char.strip()) for linha in mem_secao.splitlines()]
    ) + " }"

    asm_secao = transformar_labels_e_exit_em_linhas(asm_secao)


    # Formatar o asm e o resultado para .lean
    lean_output = f"""{cabecalho.strip()}
    \ndef memory :=
    createStackMemory 0 emptyMemory  \n{mem_formatada}
    \n#eval exeConformance {{exe|\n# Copyright Big Switch Networks Inc
# SPDX_License_Identifier Apache_2_0  \nasm  \n{asm_secao}  \nresult   \n{result_secao}   \n}} memory
    """
    return lean_output.strip()


In [31]:

def ler_arquivos_data_e_transformar(diretorio):
    # Verificar se o diretório existe
    if not os.path.exists(diretorio):
        print("O diretório especificado não existe.")
        return
    
    # Criar diretório para os arquivos .lean, se não existir
    destino = os.path.join(diretorio, "arquivos_lean")
    os.makedirs(destino, exist_ok=True)

    # Percorrer arquivos no diretório
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith(".data"):  # Verifica extensão .data
            caminho_arquivo = os.path.join(diretorio, arquivo)
            
            # Ler conteúdo como string
            with open(caminho_arquivo, "r", encoding="utf-8") as f:
                conteudo = f.read()
                conteudoLean = parseTests(conteudo)
            
            # Transformar nome do arquivo .data em .lean
            nome_arquivo_lean = os.path.splitext(arquivo)[0] + ".lean"
            caminho_arquivo_lean = os.path.join(destino, nome_arquivo_lean)
            
            # Salvar string como arquivo .lean
            with open(caminho_arquivo_lean, "w", encoding="utf-8") as f:
                f.write(conteudoLean)
            
            print(f"Arquivo {arquivo} transformado em {nome_arquivo_lean} com sucesso!")

In [32]:
diretorio_entrada="tests\data"
ler_arquivos_data_e_transformar(diretorio_entrada)

Arquivo add.data transformado em add.lean com sucesso!
Arquivo add64.data transformado em add64.lean com sucesso!
Arquivo alu-arith.data transformado em alu-arith.lean com sucesso!
Arquivo alu-bit.data transformado em alu-bit.lean com sucesso!
Arquivo alu64-arith.data transformado em alu64-arith.lean com sucesso!
Arquivo alu64-bit.data transformado em alu64-bit.lean com sucesso!
Arquivo arsh32-imm-high.data transformado em arsh32-imm-high.lean com sucesso!
Arquivo arsh32-imm-neg.data transformado em arsh32-imm-neg.lean com sucesso!
Arquivo arsh32-imm.data transformado em arsh32-imm.lean com sucesso!
Arquivo arsh32-reg-high.data transformado em arsh32-reg-high.lean com sucesso!
Arquivo arsh32-reg-neg.data transformado em arsh32-reg-neg.lean com sucesso!
Arquivo arsh32-reg.data transformado em arsh32-reg.lean com sucesso!
Arquivo arsh64-imm-high.data transformado em arsh64-imm-high.lean com sucesso!
Arquivo arsh64-imm-neg.data transformado em arsh64-imm-neg.lean com sucesso!
Arquivo arsh